In [341]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats

In [285]:
df_ = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv").set_index('States')

In [286]:
df = sm.add_constant(df_)

In [369]:
'''
a = (56, 128.6, 12, 123.8, 64.34, 78, 763.3)
b = (1.1, 2.9, 4.2)
#Use the trim keyword to perform a trimmed (Yuen) t-test. For example, using 20% trimming, trim=.2, the test will reduce the impact of one (np.floor(trim*len(a))) element from each tail of sample a. It will have no effect on sample b because np.floor(trim*len(b)) is 0.

scipy.stats.ttest_ind(a, b, trim=.2).pvalue
#Ttest_indResult(statistic=3.4463884028073513,
                #pvalue=0.01369338726499547)
'''

0.01369338726499547

In [384]:
import scipy.stats as st

def f_test(x, y, alt="two_sided"):
    """
    Calculates the F-test.
    :param x: The first group of data
    :param y: The second group of data
    :param alt: The alternative hypothesis, one of "two_sided" (default), "greater" or "less"
    :return: a tuple with the F statistic value and the p-value.
    """
    df1 = len(x) - 1
    df2 = len(y) - 1
    f = x.var() / y.var()
    if alt == "greater":
        p = 1.0 - st.f.cdf(f, df1, df2)
    elif alt == "less":
        p = st.f.cdf(f, df1, df2)
    else:
        # two-sided by default
        # Crawley, the R book, p.355
        p = 2.0*(1.0 - st.f.cdf(f, df1, df2))
    return f, p

In [389]:

#nested functions (kind of like classes, get access to parent functions members).
def filter_df(x,y):
    
    def print_df(aboveMedian,belowMedian):
        
        data = df[[y,*x]].sort_values(kind="quicksort", by=y, ascending=True)
        #abovePos = [i for i, item in enumerate(data.columns) if item in aboveMedian]
        #belowPos = [i for i, item in enumerate(data.columns) if item in belowMedian]

        print(data.describe())
        med = pd.DataFrame(np.median(data,axis=0)).T
        med.columns = data.columns
        
        filtered = data.copy()
        
        filtered = filtered.iloc[pd.DataFrame(np.array(filtered[list(aboveMedian)])>=np.array(med[list(aboveMedian)])).replace(False,np.NaN).dropna().index]
        
        filtered = filtered.iloc[pd.DataFrame(np.array(filtered[list(belowMedian)])<=np.array(med[list(belowMedian)])).replace(False,np.NaN).dropna().index]
        '''
        A=notfiltered
        B=filtered
        H0: A>=B
        H1: B>A
        '''
        sp = (scipy.stats.ttest_ind(filtered[y],data[y],equal_var=False))
        print(sp.pvalue/2)
        print(scipy.stats.ttest_1samp(a=filtered[y],popmean=np.mean(data[y])))
        print("f/variance test:",f_test(filtered[y],data[y]))
        #if(sp.pvalue/2<.05):
            #print("reject null hypothesis, B(filter)>A")
        #df_ = len(filtered[y]) + len(data[y]) - 2
        
        plt.plot(data[y])
        plt.plot(filtered[y])
        plt.show()
        
        q_s_ = data[y].quantile([.02,.09,.25,.5,.75,.91,.98])
        q_s_n = filtered[y].quantile([.02,.09,.25,.5,.75,.91,.98])
        
        plt.plot(q_s_)
        plt.plot(q_s_n)
        plt.show()
        #fig = mpf.figure(figsize=(10, 7), style=s)
        fig = plt.figure(figsize=plt.figaspect(2.))
        fig.suptitle('boxplots')
        #fig = Figure()
        #FigureCanvas(fig) # not needed in mpl >= 3.1
        ax1 = fig.add_subplot(2,1,1)
        ax2 = fig.add_subplot(2,1,2)

        ax1.boxplot(data[y])
        ax2.boxplot(filtered[y])

        plt.show()
        data[y].hist()
        filtered[y].hist()
        plt.show()
        print(filtered.describe())
        
        return(filtered)
    
    aboveMedian = widgets.SelectMultiple(
        options=df[list(x)].columns,
        description='AboveMedian',
        disabled=False
    )
    
    belowMedian = widgets.SelectMultiple(
        options=df[list(x)].columns,
        description='BelowMedian',
        disabled=False
    )
    
    interact(print_df,aboveMedian=aboveMedian,belowMedian=belowMedian)
    
def derive_xnames(y):
    x_ = widgets.SelectMultiple(
        options=df.columns[~df.columns.isin([y])],
        value=list(df.columns[~df.columns.isin([y])]),
        disabled=False
    )
    
    interact(filter_df,x=x_,y=y)

In [390]:

y=widgets.Select(options=df_.columns,disabled=False)

interact(derive_xnames,y=y)


interactive(children=(Select(description='y', options=('Poverty', 'Infant Mort', 'White', 'Crime', 'Doctors', …

<function __main__.derive_xnames(y)>